In [535]:
import Base.Threads.@spawn

using NPZ

include("WGR-MCMClib/WGR.jl")
using .WGR

include("WGR-MCMClib/Alfheim.jl")
using .Alfheim

import Base.flush
using Distributions

using Formatting
using Test

# buf = WGR.BlockBuffer(32 * Threads.nthreads())
buf = WGR.BlockBuffer(128)
setBlocks(x,y,z,b) = WGR.setBlocks!(buf, x, y, z, b)
flush() = WGR.flush!(buf)
refreshNd(a,p,f = false) = WGR.refreshNd!(buf,a,p,f)

refreshNd (generic function with 2 methods)

In [536]:
function testBFS()
    Ntest = 10
    Mtest = 1000

    vox = rand(1:2, Ntest, Ntest, Ntest)
    dist = fill(1000, Ntest, Ntest, Ntest)
    dist[3, 2, 1] = 0
    dirc = zeros(UInt8, Ntest, Ntest, Ntest)
    visited = zeros(Bool, size(vox)...)
    cost = ones(2, 2, 6)
    cost[1,1,:] .= 1
    cost[1,2,:] .= 5
    cost[2,1,:] .= 1
    cost[2,2,:] .= 5
    minB = [2,2,2]
    maxB = [-1,-1,-1]
    starts = [[3,2,1]]

    initialSum = sum(dist[2:end-1, 2:end-1, 2:end-1])

    diff = Alfheim.BFS!(dist, dirc, vox, cost, starts, visited; minB = minB, maxBFromBoundary = maxB)
    # @show dist
    @show sum(dist[2:end-1, 2:end-1, 2:end-1])

    for i in 1:Mtest
        p = rand(2:(Ntest-1), 3)
        vox[p...] = 2 - vox[p...] + 1 # Flip
        starts = [p]
        diff += Alfheim.BFS!(dist, dirc, vox, cost, starts, visited; minB = minB, maxBFromBoundary = maxB)
    end

    @show sum(dist[2:end-1, 2:end-1, 2:end-1])

    # Re-calculate
    dist = fill(1000, Ntest, Ntest, Ntest)
    dist[3, 2, 1] = 0
    dirc = zeros(UInt8, Ntest, Ntest, Ntest)
    starts = [[3,2,1]]
    Alfheim.BFSbad!(dist, dirc, vox, cost, starts, visited; minB = minB, maxBFromBoundary = maxB)
    nextSum = sum(dist[2:end-1, 2:end-1, 2:end-1])

    @show dist[2:end-1, 2:end-1, 2:end-1]
    @show @test initialSum + diff == nextSum
end

testBFS (generic function with 1 method)

In [537]:
function pad(arr, padding = 1, content = 0)
    padded = fill(content, size(arr) .+ padding * 2)
    padded[1+padding:end-padding, 1+padding:end-padding, 1+padding:end-padding] = arr
    return padded
end

pad (generic function with 3 methods)

In [538]:
monu10 = npzread("../Python/AlfheimMCMC/alfheimTest.npy")
monu10 = pad(monu10, 1, 0)
shape = size(monu10)
println(shape)

(18, 20, 18)


In [539]:
# Pre-process raw monu10 into palette'd monu10
#Block Index
function blockIndex(blocks)
    blockDict = Dict{UInt32, Array{Int64}}(0 => [1, 0])
    blockRep = [0]
    for b in monu10
        if !haskey(blockDict, b)
            blockDict[b] = [length(blockDict) + 1, 0]
            push!(blockRep, b)
        end
        blockDict[b][2] += 1
    end
    
    return blockDict, blockRep
end

bDict, bRep = blockIndex(monu10)

pmonu10 = [Alfheim.Block(bDict[b][1]) for b in monu10]

# Hack
pmonu10[pmonu10 .!= 1] .= 2
bDict = filter(b -> last(b)[1] <= 2, bDict)
@show bDict
bRep = convert(Array{WGR.Block}, bRep[1:2])

Nb = length(bDict)
bTotal = sum([b[2] for b in values(bDict)])
bDist = Categorical([bDict[bRep[b]][2] / bTotal for b in 1:Nb])

bDict = Dict{UInt32, Array{Int64}}(0x00000000 => [1, 5207], 0x99ccccff => [2, 1273])


Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.8035493827160494, 0.1964506172839506])

In [540]:
refreshNd(bRep[pmonu10], [1, 65, 1], false)

0

In [541]:
# Parameters
# Ns = [2]       # Pattern size
# Nweight = [500.0]
step = 1    # Pattern stride
mcsize = 64 # MCMC size (XZ)
mcheight = 12 # MCMC height
eps = 3e-4  # Epsilon
# targetPosThreads = [[100 + 40 * , 65, 1]] # Where to put the mcmc
nIters = 1400000

bDistλ = mcsize^2 * mcheight * 6
# bDistλ = 40

Tstart = 12.0
Tend = 5.0

NDR = 2
padding = 2

2

In [542]:
function initialize()
    # arr = [1 for x in 1:mcsize, y in 1:mcsize, z in 1:mcsize]
    arr = Alfheim.Block[rand(bDist) for x in 1:mcsize, y in 1:mcheight, z in 1:mcsize]
    # pst = [30, 40, 7]
    # arr = pmonu10[[pst[i]:pst[i]+mcsize for i in 1:3]...]
    arr = pad(arr, padding, 1)
    # arr = copy(pmonu10)

    # walker = Alfheim.RandomBlockProposal(
    walker = Alfheim.RandomBlockProposalAlwaysDifferent(
        # bDist,
        Categorical([1 / Nb for i in 1:Nb]),
        1,
        [1, 1, 1] .+ padding,
        [size(arr)[i] for i in 1:3] .- padding
    )

    # Readin patterns
    patterns = [
        Alfheim.PatternMarginals(2),
        Alfheim.Patterns(3),
    ]
    pWeights = [
        500.0,
        1.03
    ]
    @time [Alfheim.readSlices!(p, pmonu10) for p in patterns]

    ############# Create Paths energy

    # Costs
    costs = zeros(Int, Nb, Nb, Nb, Nb, 6)
    
    costs[1:Nb, 1:Nb, 1, 1, 1:6] .= 10 # -> Void
    costs[1:Nb, 1:Nb, 2:Nb, 1:Nb, 1:6] .= 10 # -> Solid
    
    costs[1:Nb, 1:Nb, 1, 2:Nb, 1:2] .= 10 # -> Road, vertical (should not appear)
    costs[1:Nb, 1:Nb, 1, 2:Nb, 3:6] .= 2 # -> Road, horizontal
    
    costs[1, 2:Nb, 1, 1, 1:6] .= 3 # Road -> Void (stairs)
    costs[1, 1, 1, 2:Nb, 1:6] .= 3 # Void -> Road (stairs)
    
    # Test cost
    # costs = zeros(Int, Nb, Nb, 6)
    # costs[1:Nb, 1:Nb, 1:6] .= 1
    
    @show findall(x -> x>0, costs .== 0)

    # Arrays
    dist = fill(Int(2^30), size(arr))
    dirc = ones(UInt8, size(arr)...)

    # Entrance
    entranceSize = mcsize ÷ 3
    entranceStart = (mcsize + 2 * padding - entranceSize) ÷ 2 + 1
    entranceEnd = entranceStart + entranceSize

    # 4 Entries
    starts = []
    
    dist[entranceStart:entranceEnd, 1:1 + padding, 1:padding] .= 0
    starts = [starts; [[x, 1 + padding, padding] for x in entranceStart:entranceEnd]]
    arr[entranceStart:entranceEnd, 1:padding, 1:padding] .= 2
    
    dist[1:padding, 1:1 + padding, entranceStart:entranceEnd] .= 0
    starts = [starts; [[padding, 1 + padding, x] for x in entranceStart:entranceEnd]]
    arr[1:padding, 1:padding, entranceStart:entranceEnd] .= 2
    
    dist[entranceStart:entranceEnd, 1:1 + padding, end:-1:end-padding+1] .= 0
    starts = [starts; [[x, 1 + padding, size(arr)[3]-padding+1] for x in entranceStart:entranceEnd]]
    arr[entranceStart:entranceEnd, 1:padding, end:-1:end-padding+1] .= 2
    
    dist[end:-1:end-padding+1, 1:1 + padding, entranceStart:entranceEnd] .= 0
    starts = [starts; [[size(arr)[1]-padding+1, 1 + padding, x] for x in entranceStart:entranceEnd]]
    arr[end:-1:end-padding+1, 1:padding, entranceStart:entranceEnd] .= 2

    @show [1, 1, 1] .+ padding

    # pathsEnergy = Alfheim.Paths(costs, starts, [[0,0,0]], walker.boundsMin, walker.boundsMax, dist, dirc)
    pathsEnergy = Alfheim.Paths(costs, starts, [[0,0,0], [0,-1,0]], walker.boundsMin, walker.boundsMax, dist, dirc)
    pathsWeight = 0.07

    ############# Create gravity paths energy

    # Gravity / support
    costs = zeros(Int, Nb, Nb, 6)
    costs[2:Nb, 2:Nb, 1] .= 5 # To solid, up
    costs[2:Nb, 2:Nb, 2] .= 1 # To solid, down
    costs[2:Nb, 2:Nb, 3:6] .= 3 # To solid, left / right / forward / back
    costs[2:Nb, 1, 1:6] .= 100 # To air
    costs[1, 1, 1:6] .= 1 # air -> air

    # Arrays
    dist = fill(Int(2^30), size(arr))
    dirc = ones(UInt8, size(arr)...)

    # Entrance
    dist[:, 1:padding, :] .= 0
    starts = []
    for x in 1:size(arr)[1]
        for y in 1:padding
            for z in 1:size(arr)[3]
                push!(starts, [x, y, z])
            end
        end
    end

    # Fill init voxels
    arr[:, 1:padding, :] .= 2

    pathsEnergyG = Alfheim.Paths(costs, starts, [[0,0,0]], walker.boundsMin, walker.boundsMax, dist, dirc)
    pathsWeightG = 0.13

    ###############################################################################################

    mstate = Alfheim.createMCMCState(

        # Initial voxels
        arr,

        # Energies
        Alfheim.VoxelGibbsDistribution(
            [
                Alfheim.BlockMargins(bDist.p),
                patterns...,
                pathsEnergy,
                pathsEnergyG,
            ],
            [
                bDistλ,
                pWeights...,
                pathsWeight,
                pathsWeightG,
            ]
        ),

        # Walker
        walker
    )

    opt = Alfheim.Metropolis()
    # opt = Alfheim.ReversibleDRMetropolis(NDR)
    
    return mstate, opt
end

initialize (generic function with 1 method)

In [543]:
methods(Alfheim.initSlices!)

# 2 methods for generic function "initSlices!":
[1] initSlices!(sW::Main.Alfheim.PatternMarginals, nSlice) in Main.Alfheim at D:\RP_Games\UnityProj\WGRDemo\Julia\WGR-MCMClib\Slices.jl:217
[2] initSlices!(sW::Main.Alfheim.Patterns, nSlice::Array{UInt8, 3}) in Main.Alfheim at D:\RP_Games\UnityProj\WGRDemo\Julia\WGR-MCMClib\Slices.jl:115

In [544]:
function generate(mstate, opt, targetPos)
    
    # targetPos = targetPosThreads[threadID]
    
    flush()
    height = size(mstate.voxels)[2]
    # for p in starts
    #     setBlocks((p + targetPos .- 1)..., 0xff553388)
    # end
    flush()

    Alfheim.init!(mstate)
    rejected = 0

    sTime = time_ns()

    accept_count = zeros(NDR,)
    flush(stdout)
    
    # Show initial voxels
    # refreshNd(bRep[mstate.voxels], targetPos, true)
    for x in 1:size(mstate.voxels)[1], y in 1:size(mstate.voxels)[2], z in 1:size(mstate.voxels)[3]
        p = [x, y, z]
        b = mstate.voxels[p...]
        setBlocks((p + targetPos .- 1)..., WGR.darken(bRep[b], ((height - p[2]) / height) * 0.75))
    end
    
    # MCMC Iterations
    for it in 1:nIters

        T = (Tstart - Tend) * (1 - (it / nIters)) + Tend
        accepted, ds = Alfheim.step!(opt, mstate; temperature = T)

        if (accepted > 0)
            accept_count[accepted] += 1
            for (p, b) in zip(ds.pos, ds.blk)
                setBlocks((p + targetPos .- 1)..., WGR.darken(bRep[b], ((height - p[2]) / height) * 0.75))
            end
        else
            rejected += 1
        end

        if it % (div(nIters, 100)) == 0
            eTime = time_ns()

            iter_s = div(nIters, 100) / (eTime - sTime) * 1e9
            ETA = (nIters - it) / iter_s

            printfmt("Iter {:8d} @ {:7.4f} | E = {:8.1f} (E_path = {:8.1f}) | {:7.1f} it / s, {:5.1f}s ETA, aR {:5.1f}% ({:5.1f}%: ", 
                it, T, sum(mstate.p.E), mstate.p.E[end], iter_s, ETA, 
                (it - rejected) / it * 100.0,
                (sum(accept_count)) / div(nIters, 100) * 100.0)
            [printfmt("{:5d} ", accept_count[i]) for i in 1:NDR]
            print(")\n")

            flush()
            flush(stdout)

            sTime = time_ns()
            # last_accd = it - rejected
            accept_count = zeros(NDR,)
        end

        # flush(stdout)
    end

    flush()
end

generate (generic function with 1 method)

In [545]:
mstate = nothing
opt = nothing

function work(targetPos)
    m, o = initialize()
    
    mstate = m
    opt = o
    
    generate(m, o, targetPos)
    return m, o
end

work (generic function with 1 method)

In [546]:
# tasks = []

# for ti in 1:Threads.nthreads()
#     push!(tasks, @spawn work([100 + 40 * ti, 65, 0]))
# end
mstate, opt = work([100, 65, 0])

  0.149357 seconds (420.54 k allocations: 25.483 MiB, 14.83% gc time, 91.41% compilation time)
findall((x->begin
            #= In[542]:47 =#
            x > 0
        end), costs .== 0) = CartesianIndex{5}[]
[1, 1, 1] .+ padding = [3, 3, 3]
sW.reference[1:10] = [26.46875, 3.30859375, 16.54296875, 6.6171875, 36.39453125, 3.30859375, 3.30859375, 175.35546875, 1038.8984375, 16.54296875]
sW.state.MCMCslicesCount[1:10] = [5, 33, 10, 4, 111, 25, 123, 2448, 31, 0]
sW.state.E = 115.18385481071958
sW.MCMCnumSlices = 60984
Iter    14000 @ 11.9300 | E = 1037775.3 (E_path = 121404.7) |   270.0 it / s, 5134.0s ETA, aR  34.4% ( 34.4%:  3074  1736 )
Iter    28000 @ 11.8600 | E = 984699.9 (E_path =  99470.5) |   288.6 it / s, 4754.5s ETA, aR  33.3% ( 32.2%:  2807  1695 )
Iter    42000 @ 11.7900 | E = 945127.7 (E_path =  84372.5) |   303.6 it / s, 4473.5s ETA, aR  32.2% ( 29.9%:  2468  1724 )
Iter    56000 @ 11.7200 | E = 909887.2 (E_path =  74026.2) |   289.4 it / s, 4644.1s ETA, aR  31.3% ( 28.7%:  

(Main.Alfheim.MCMCState1([0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01; … ; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01;;; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01; … ; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01;;; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01; … ; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01;;; … ;;; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01; … ; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01;;; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01; … ; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01;;; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01; … ; 0x02 0x02 … 0x01 0x01; 0x02 0x02 … 0x01 0x01], Main.Alfheim.VoxelDifference([[55, 3, 34]], UInt8[0x02]), Main.Alfheim.VoxelGibbsDistribution(Main.Alfheim.AbstractEnergy[Main.Alfheim.BlockMargins([0.8035493827160494, 0.1964506172839506], 73984, Main.Alfheim.L2, Main.Alfheim.BlockMarginsState([59449, 14535], 1.5244903824570454e-5), Main.Alfheim.BlockMarginsState[Main.Alfheim.BlockMarginsState([59449, 14535], 1.5244903824570454

In [547]:
# interrupt()

In [548]:
flush()

0

In [549]:
# pathsEnergy.state.dist[2:29, 9, 2:29]

In [550]:
# finaldist = copy(pathsEnergy.state.dist[2:29, 9:9, 2:29])
# @show size(finaldist .* (256^3).+ 0x000000ff)
# @show targetPos + [1, 8, 1]
# refreshNd(finaldist .* 5 * (256^3) .+ 0x000000ff, targetPos + [1, 8, 1], false)

In [551]:
# refreshNd(zeros(28,28,28), targetPos + [0,9,0], true)

In [552]:
# pathsEnergy.state

In [553]:
function recalcE()
    # Gravity / support
    costs = zeros(Int, Nb, Nb, 6)
    costs[2:Nb, 2:Nb, 1] .= 5 # To solid, up
    costs[2:Nb, 2:Nb, 2] .= 1 # To solid, down
    costs[2:Nb, 2:Nb, 3:6] .= 3 # To solid, left / right / forward / back
    costs[2:Nb, 1, 1:6] .= 100 # To air
    costs[1, 1, 1:6] .= 1 # air -> air

    # Arrays
    dist = fill(Int(2^30), mcsize+2*padding, mcheight+2*padding, mcsize+2*padding)
    dirc = ones(UInt8, mcsize+2*padding, mcheight+2*padding, mcsize+2*padding)

    # Entrance
    dist[:, 1:padding, :] .= 0
    starts = []
    for x in 1:mcsize+2*padding
        for y in 1:padding
            for z in 1:mcsize+2*padding
                push!(starts, [x, y, z])
            end
        end
    end

    mstate.p.energy[end].cost = costs
    mstate.p.energy[end].state.dist = dist
    mstate.p.energy[end].state.dirc = dirc
    mstate.p.energy[end].starts = starts
    mstate.p.energy[end].minBounds = mstate.walker.boundsMin
    mstate.p.energy[end].maxBounds = mstate.walker.boundsMax

    mstate.p.energy[end](mstate)
end

recalcE (generic function with 1 method)

In [554]:
# colors = @. (min(mstate.p.energy[end].state.dist[3:end-2, 3:end-2, 3:end-2] * 10, 255) .* 0x00010000 .+ 0x000000ff) .* (mstate.voxels[3:end-2, 3:end-2, 3:end-2] .- 1)
colors = @. (min(mstate.p.energy[end-1].state.dist[1:end, 3:end-1, 1:end] * 2, 255) .* 0x01000000 .+ 0x000000ff) .* (mstate.voxels[1:end, 2:end-2, 1:end] .- 1)
# colors = @. (min((mstate.p.energy[end].state.dirc[3:end-2, 3:end-2, 3:end-2] == 2) * 255, 255) .* 0x01000000 .+ 0x000000ff) .* (mstate.voxels[3:end-2, 3:end-2, 3:end-2] .- 1)

# mstate.p.energy[end].state.dist[3:end-2, 3:end-2, 3:end-2]
size(colors)

(68, 13, 68)

In [555]:
# refreshNd(colors, [100, 66, 0], true)
# refreshNd(colors, [100, 65, 0] .+ 2, true)
# refreshNd(bRep[mstate.voxels], [100, 65, 0], true)

In [556]:
using Serialization
Serialization.serialize("ElfTown.mstate", mstate)